# multi frame'spine instances by 2d model
1. ##!!--------------------!!##
   
   some paras which need be set by users 
2. ##%%-------------------%%##
   
   some edit need be down by users 

In [3]:
# import library 导入库文件
import sys
import tifffile
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
from skimage.io import imsave
from skimage.morphology import remove_small_objects
InteractiveShell.ast_node_interactivity = 'last_expr'
from tqdm import tqdm
import numpy as np
import os


sys.path.append("..")
sys.path.append("..//..")

from IPython.core.interactiveshell import InteractiveShell
from skimage.filters import threshold_otsu, threshold_yen
from skimage.filters.thresholding import (threshold_isodata, threshold_li,
                                          threshold_local, threshold_mean,
                                          threshold_minimum,
                                          threshold_multiotsu,
                                          threshold_niblack, threshold_otsu,
                                          threshold_triangle, threshold_yen)
from skimage.io import imsave
from skimage.morphology import remove_small_objects
from skimage.segmentation import expand_labels, watershed
from utils import file_base
import spinelib
from spinelib import cflow, imgio, seg
from spinelib.cflow.blob import find_spheres, peakfilter, sphere_log
from spinelib.cflow.localthreadhold import local_threshold
from spinelib.imgio import napari_base
from spinelib.seg import segment, track,unetseg
from spinelib.utils import npixel
from spinelib.utils.measure import label_statics
from spinelib.utils.spine_struct import spines_distance
from spinelib.imgio import imgio_base
from spinelib.imgio.imgio_base import savelabel,savepr
from utils import file_base,yaml_config
from train.trainers import predict
# InteractiveShell.ast_node_interactivity = 'last_expr'
methods = cflow.localthreadhold.thmethods


c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


No Fortran modules found, falling back on Python implementation.
Did you run `python3 setup.py install`?


c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] 找不到指定的程序。
  warn(f"Failed to load image Python extension: {e}")


In [4]:
#basic fun

def remove_den(imgs,unetmodel,cval=140):
    denmasks,_=predict.predict_time_img(unetmodel,imgs)
    imgs=imgs.copy()
    imgs[denmasks==1]=cval
    return imgs
def remove_small_lable(mask,thsize):
    labs=measure.unique_labs(mask)
    newmask=np.zeros_like(mask)
    for lab in labs:
        mm=mask==lab
        mask2=remove_small_objects(mm,thsize)
        if not mask.any():
            mask2=mm
        newmask[mask2]=lab
    return newmask   
def th_img(imgs,ls,binary_func):
    global_th=binary_func(imgs[0])
    lss=np.zeros_like(imgs,dtype=np.uint16)

    for i in range(len(imgs)):
        img=imgs[i]
        adth=img>global_th
        # denmask,_=predict.predict_single_img(unetmodel,img)
        l=ls*adth
        l[l<2]=0
        l=remove_small_lable(l,6)
        lss[i]=l
        # lss[i][denmask==1]=0
    lss[lss<2]=0
       
    return lss




In [5]:
# super params
#-----------------------#
#   paras  #
#-----------------------#
# 设置载入2D/3D  参数
paras_2D=dict(
    axes="txy", #2D
    sigma=(2,2), # 2D y,x
    steps=2, # caculate stick gradient step lenth 1-4, larger more error , smaller than spine diameter
    lambda1=1,
    lambda2=1.5, # lambda2 need large than lambda1
    sigma_hessian = 4,
    sigma_supp = 100,
    countth=10*20 ,# fiber largest pixels, include z stack, that to say lenth*stackn
    smallobj=200,
    spine_radius=35, # for iteration to growth area
    min_radius=3,
    maxspinesize=600,
    spotrange=[3,11],
    binary_func=threshold_li, # local_threshold  #threshold_li
    use_sobel=False,
    search_box=[25,25],
    keep_time=5,
    peak_use_gaussian=True,
    spine_min_size_th=4, 
    searchbox=[5,5],
)
paras_3D=dict(
    axes="tzxy", #
    sigma=(2,2,2), # y,x
    steps=2, # caculate stick gradient step lenth 1-4, larger more error , smaller than spine diameter
    lambda1=1,
    lambda2=1.5, # lambda2 need large than lambda1
    sigma_hessian = 4,
    sigma_supp = 100,
    countth=10*20 ,# fiber largest pixels, include z stack, that to say lenth*stackn
    smallobj=200,
    spine_radius=35, # for iteration to growth area
    min_radius=3,
    maxspinesize=200,
    spotrange=[3,11],
    binary_func=threshold_li, # local_threshold  #threshold_li
    use_sobel=False,
    search_box=[25,25],
    keep_time=5,
    peak_use_gaussian=True,
    spine_min_size_th=4, 
    searchbox=[5,5,5]
)


In [6]:
#-----------------------#
#   model loader  #
#-----------------------#
# 选择使用的配置文件,用于载入预测模型和参数

configfile=r"..\config\defalut_2d_ins_ERF.yaml"
default_configuration=yaml_config.YAMLConfig(
        configfile
        )
premodel=r"F:\code\zly\myspine-20230606\DATA\2d_ins_ERF\model\ep043-loss0.956.pth"
# modelpath=r"D:\code\myspine\models\defaultmodel\actin-modelep006-loss0.097.h5"
# modelpath=r"D:\code\myspine\models\defaultmodel\morph-modelep200-loss0.133.h5"
# modelpath=r"D:\code\test-dataset\model\2D\segep064-loss0.125.h5"

predictmodel = predict.Predict()

predictmodel.setting(default_configuration,use_gpu=False)
predictmodel.load_weight(False,premodel)

== Configure file dir f:\code\zly\myspine-20230606\config
== Configure file path f:\code\zly\myspine-20230606\config\defalut_2d_ins_ERF.yaml
== Data root F:\data\dataset\myspine-dataset\2D-morph-seg
== Train root f:\code\zly\myspine-20230606\config\../DATA/2d_ins_ERF
== crop_path f:\code\zly\myspine-20230606\config\../DATA/2d_seg/data
f:\code\zly\myspine-20230606\DATA\2d_ins_ERF\log\train_info.log
device: cpu
================device enable state============
use gpu	: 	 False
gpu number		: 	 0
use cpu	: 	 True
cpu number		: 	 12
Creating branched erfnet with [3, 3] classes
initialize last layer with size:  torch.Size([16, 3, 2, 2])
============= assign xym =============


2023-06-08 18:54:47,652 - ..\train\trainers\predict.py[line:114] - INFO: 
==============LOAD PRETRAINED model==============
F:\code\zly\myspine-20230606\DATA\2d_ins_ERF\model\ep043-loss0.956.pth



In [55]:
#-----------------------#
#   data loader  #
#-----------------------#
# 选择要分析的图像
filename=r"F:\data\LNP2DATA\QL\QL2Dt\Stablized tboaltp 20180128n2.tif"

#-----------------------#
#   Output file path  # 
#-----------------------#
# 选择要保存的文件夹
savedir=r"F:\data\dataset\myspine-dataset2\result-time"
file_base.create_dir(savedir)
#-----------------------#
#   Loading image  #
#-----------------------#
#-- image width and height
imgs = imgio.ski_imread(filename)
imgio_base.show_img_info(imgs)
ndim=imgs.ndim
ftime=False
if(ndim==4):
    wx,wy=imgs.shape[2:4]
else:
    wx,wy=imgs.shape[1:3]

#-----------------------#
#   Output file path  #
#-----------------------#
# 选择要保存的文件夹

file_base.create_dir(savedir)

#-----------------------#
#   Output file list  #
#-----------------------#
ddir,shotname,suffix=file_base.split_filename(filename)
ddir,shortname,suffix=file_base.split_filename(filename)
savedir=os.path.join(savedir,shortname)
file_base.create_dir(savedir)
# 将会生成的文件
imgfile=os.path.join(savedir,shotname+".tif")
spineprfile=os.path.join(savedir,shotname+"-spinepr.tif")
denprfile=os.path.join(savedir,shotname+"-denpr.tif")
maskfile=os.path.join(savedir,shotname+"-mask.tif")
spinefile=os.path.join(savedir,shotname+"-spine.tif")
denfile=os.path.join(savedir,shotname+"-den.tif")
skeletonfile=os.path.join(savedir,shotname+"-skeletonden.tif")
mipimgfile=os.path.join(savedir,"tMIP-"+shotname+".tif")
mipmaskfile=os.path.join(savedir,"tMIP-"+shotname+"-mask.tif")
mipspineprfile=os.path.join(savedir,"tMIP-"+shotname+"-spinepr.tif")
mipdenprfile=os.path.join(savedir,"tMIP-"+shotname+"-denpr.tif")
mipspinefile=os.path.join(savedir,"tMIP-"+shotname+"-spine.tif")
mipdenfile=os.path.join(savedir,"tMIP-"+shotname+"-den.tif")

areafile=os.path.join(savedir,shotname+"_area.csv")
meanfile=os.path.join(savedir,shotname+"_mean_inten.csv")
# 如果是3D的图像
zareafile=os.path.join(savedir,shotname+"_zarea.csv")
zmeanfile=os.path.join(savedir,shotname+"_zmean_inten.csv")




#=======================#
#   load paras  #
#=======================#
paras=paras_2D if ndim==2 else paras_3D
binary_func=paras["binary_func"]
spine_min_size_th=paras["spine_min_size_th"]
min_radius=paras["min_radius"]
searchbox=paras["searchbox"]
maxspinesize=paras["maxspinesize"]

image shape : (80, 512, 512)
image dtype : uint16
image ndim : 3


# PART ONE - predict by model


1. Let's see predict spine and dendritc probability 

need process each frame by network model,maybe slowly,please be patient

In [46]:
#--- Predict ---#
mip_img=np.max(imgs,axis=0)
#--- Predict ---#
from csbdeep.utils import normalize
from spinelib.seg import unetseg

model=predictmodel.model
# print(img.shape)
# img=normalize(img)
mip_ins,mip_prob,mip_mask=model.off_predict(mip_img)
mip_bgpr,mip_denpr,mip_spinepr=mip_prob
#mask,spinepr,denpr,bgpr=unetseg.predict_single_img(model,mip_img)

# spine_label=unetseg.instance_unetmask_bypeak(spinepr,mask,searchbox[1:],min_radius,spinesize_range=[4,800])
#spine_label=unetseg.instance_unetmask_by_border(spinepr,mask==2,bgpr,0.5,spinesize_range=[8,800])



============= assign xym =============


In [47]:
#--- View  MIP Predict Result ---#
viewer=imgio.plot.imshow3d(I=mip_img,
    Is_overlay=[imgs,mip_spinepr,mip_denpr],#bgpr
    name_Is=[   "imgs","spinepr","denpr"],
    cmap_Is=[   "gray","magenta",  "green"],
    label_lay=[mip_mask,mip_ins],
    label_lay_name=["mask","spine"]   )

In [48]:
#imgs=imgs[:10]
def predict_time_imgs(model,imgs):
    # print("img shape : ",img.shape)
    masks=[]
    preds=[]
    inss=[]
  
    n_frames=imgs.shape[0]
    for i in tqdm(range(n_frames), desc='Processing'):
        img=imgs[i]
        ins,prob,mask=model.off_predict(img)
        masks.append(mask)
        preds.append(prob)
        inss.append(ins)
        
    masks=np.array(masks)
    masks=np.squeeze(masks)    
    preds=np.array(preds)
    preds=np.squeeze(preds) 
    inss=np.array(inss)
    inss=np.squeeze(inss) 
    preds=preds.transpose((1,0,2,3))
    return masks,preds,inss

masks,preds,inss=predict_time_imgs(model,imgs)

Processing:   0%|          | 0/80 [00:00<?, ?it/s]

============= assign xym =============


Processing:   1%|▏         | 1/80 [00:01<01:22,  1.04s/it]

============= assign xym =============


Processing:   2%|▎         | 2/80 [00:01<01:17,  1.01it/s]

...

Processing:  98%|█████████▊| 78/80 [01:17<00:02,  1.00s/it]

============= assign xym =============


Processing:  99%|█████████▉| 79/80 [01:18<00:01,  1.00s/it]

============= assign xym =============


Processing: 100%|██████████| 80/80 [01:19<00:00,  1.01it/s]


In [49]:
#--- View  Predict Result ---#
viewer=imgio.plot.imshow3d(I=imgs,
    Is_overlay=[mip_img,preds[2],preds[1],preds[0]],
    name_Is=[   "mip_img","spinepr","denpr","bgpr"],
    cmap_Is=[   "gray","magenta",  "green","gray"],
    label_lay=[masks,inss,],
    label_lay_name=["masks","mip_mask"]   )

view  predict result

In [50]:
#--- choose Strtage -----#
# one : t-MIP
# two : btrack
# ------------------     -#
mip_ins,_=segment.resortseg(mip_ins,2)
mip_instance_spines=[mip_ins*(m>1) for m in masks]
instance_spines_bymip=np.array(mip_instance_spines)
viewer=imgio.plot.imshow3d(I=imgs,
    Is_overlay=[mip_img],
    name_Is=[   "mip_img"],
    cmap_Is=[   "gray"],
    label_lay=[masks,mip_ins,instance_spines_bymip],
    label_lay_name=["masks","mip_spine","spines"]   )

In [37]:
spine_ids=[103,203,131,145,220,132,167,218,126,105,234,79,280,232,211,302,237,205,276,113,143,358]
spine_ids=[280,162,205,167,130,231,145,234,237]
spine_ids=[128,37,79,18,54,125]
len(spine_ids)
keep_spines=np.zeros_like(instance_spines_bymip)
for n,lab in enumerate(spine_ids,2):
    keep_spines[instance_spines_bymip==lab]=n
keep_spines[masks==1]=1
from spinelib.utils.spine_struct import spines_distance
from skimage.filters import gaussian

shape_kvs={0:"unkownn",1:"stubby",2:"mushroom",3:"thin"}
shape_rkvs={"unkownn":0,"stubby":1,"mushroom":2,"thin":3}
viewer=imgio.plot.imshow3d(I=imgs,
    Is_overlay=[mip_img],
    name_Is=[   "mip_img"],
    cmap_Is=[   "gray"],
    label_lay=[masks,mip_mask,keep_spines],
    label_lay_name=["masks","mip_mask","spines"]   )



In [38]:

labs=set(np.unique(keep_spines))
for n,(img,labimg) in enumerate(zip(imgs,keep_spines)):
   
    corddict,linemask=spines_distance(img,labimg,[20,20],imgweight=1)
    i=0
    for lab in labs:
        i+=1
        # if i>20: break
        if lab==1: continue
        if lab not in corddict: continue
        cords,length,headlength,attach_cnt,rotamask,rotaimg,rotaden=corddict[lab]
        #print(n,lab,rotaimg)
     
        normimg=normalize(rotaimg.astype("float32"))
        normlab=normalize(gaussian(rotamask)*normimg)
        normden=normalize(gaussian(rotaden)*normimg)
        # print(normimg.shape,normlab.shape)
        # normall=np.array([normimg,rotamask,normlab,normden])
        # normall=np.array([normlab,normden])
        normall=np.array([normlab,normden,normimg])
        normall=np.transpose(normall,(1,2,0))
        #savepr(normall,os.path.join(savedir,shortname+"_"+str(lab)+"-frame-"+str(n)+"-ID-norm.tif"))
        imsave(os.path.join(savedir,"ID-"+str(lab)+"-frame-"+str(n)+"-norm.tif"),normall)
        #imsave(os.path.join(savedir,shortname+str(lab)+"-ID-norm.tif"),normall)
        #savelabel(rotamask,os.path.join(savedir,str(lab)+"-ID-mask.tif"))

In [33]:
savedir

'F:\\data\\dataset\\myspine-dataset2\\result-time\\Stablized tboaltp20180128n3'

2. Let's see spine instance segment

instance tmip img first

In [6]:
pr_corner=peakfilter(mip_spinepr,3,0,use_gaussian=True)*(mip_mask==2)#*adth
mip_spine_label=segment.label_instance_water(mip_spinepr,pr_corner,mip_mask==2, 
                                maxspinesize,searchbox=searchbox)
mip_spine_label=remove_small_objects(mip_spine_label,min_size=4)

old_mip_spine=mip_spine_label


View  Instance Result

In [8]:
##!!---- need edit , remove small segment ----!!!##
viewer=imgio.plot.imshow3d(I=imgs,
    Is_overlay=[mip_img],
    name_Is=[ "tMIPimage"],
    cmap_Is=[   "gray"],
    label_lay=[mip_mask,mip_spine_label],
    label_lay_name=["tMIP mask","tMIPspine label"]   )

mip_spine_label=viewer.layers["tMIPspine label"].data
mip_spine_label,_=segment.resortseg(mip_spine_label,2)

Contours are not displayed during 3D rendering
Contours are not displayed during 3D rendering


instance each image

In [9]:
# instance all frame
masks=all_mask==2 
all_spine_label=[mip_spine_label*mask for mask in masks]
all_spine_label=np.array(all_spine_label)



View  Instance Result

In [10]:
viewer=imgio.plot.imshow3d(I=imgs,
    Is_overlay=[mip_img],
    name_Is=[ "tMIPimage"],
    cmap_Is=[   "gray"],
    label_lay=[mip_spine_label,all_spine_label],
    label_lay_name=["tMIPspine label","spine label"]   )

all_spine_label=viewer.layers["spine label"].data

3. save all result

In [51]:

# save result

def savepr(arr,filename):
    tifffile.imwrite(
        filename,
        arr,
        imagej=True,
        photometric='minisblack',
        #metadata={'axes': 'TYX'},
    )
    print("save :",filename)
def savelabel(arr,filename):
    tifffile.imwrite(
        filename,
        arr.astype(np.uint16),
        imagej=True,
        photometric='minisblack',
        #metadata={'axes': 'TYX'},
    )
    print("save :",filename)


# save img
imsave(imgfile,imgs)
imsave(mipimgfile,mip_img)
print("save :",imgfile)
print("save :",mipimgfile)
# save pr  ,,preds[0]
savepr(preds[2],spineprfile)
savepr(preds[1],denprfile)
#mip_bgpr,mip_denpr,mip_spinepr
savepr(mip_spinepr,mipspineprfile)
savepr(mip_denpr,mipdenprfile)

# save lebel
savelabel(masks,maskfile)
savelabel(instance_spines_bymip,spinefile)
savelabel(mip_mask,mipmaskfile)
savelabel(mip_ins,mipspinefile)




C:\Users\ZLY\AppData\Local\Temp\ipykernel_35620\514942168.py:24: UserWarning: F:\data\dataset\myspine-dataset2\result-time\Stablized tboa-ltp 20180104 n3\Stablized tboa-ltp 20180104 n3.tif is a low contrast image
  imsave(imgfile,imgs)
C:\Users\ZLY\AppData\Local\Temp\ipykernel_35620\514942168.py:25: UserWarning: F:\data\dataset\myspine-dataset2\result-time\Stablized tboa-ltp 20180104 n3\tMIP-Stablized tboa-ltp 20180104 n3.tif is a low contrast image
  imsave(mipimgfile,mip_img)


save : F:\data\dataset\myspine-dataset2\result-time\Stablized tboa-ltp 20180104 n3\Stablized tboa-ltp 20180104 n3.tif
save : F:\data\dataset\myspine-dataset2\result-time\Stablized tboa-ltp 20180104 n3\tMIP-Stablized tboa-ltp 20180104 n3.tif
save : F:\data\dataset\myspine-dataset2\result-time\Stablized tboa-ltp 20180104 n3\Stablized tboa-ltp 20180104 n3-spinepr.tif
save : F:\data\dataset\myspine-dataset2\result-time\Stablized tboa-ltp 20180104 n3\Stablized tboa-ltp 20180104 n3-denpr.tif
save : F:\data\dataset\myspine-dataset2\result-time\Stablized tboa-ltp 20180104 n3\tMIP-Stablized tboa-ltp 20180104 n3-spinepr.tif
save : F:\data\dataset\myspine-dataset2\result-time\Stablized tboa-ltp 20180104 n3\tMIP-Stablized tboa-ltp 20180104 n3-denpr.tif
save : F:\data\dataset\myspine-dataset2\result-time\Stablized tboa-ltp 20180104 n3\Stablized tboa-ltp 20180104 n3-mask.tif
save : F:\data\dataset\myspine-dataset2\result-time\Stablized tboa-ltp 20180104 n3\Stablized tboa-ltp 20180104 n3-spine.tif
sa

# Part Two 
some statics , you can also use imageJ

1. load data

In [56]:
# pixelsize :
xpixel=0.16
ypixel=0.16
zpixel=1 # if no z axes ,set size=1

In [57]:
# load img
imgs=imgio.ski_imread(imgfile)
mip_img=imgio.ski_imread(mipimgfile)
# load pr
all_spinepr=imgio.ski_imread(spineprfile)
all_denpr=imgio.ski_imread(denprfile)
mip_spinepr=imgio.ski_imread(mipspineprfile)
mip_denpr=imgio.ski_imread(mipdenprfile)
# load lebel
all_mask=imgio.ski_imread(maskfile)
all_spine_label=imgio.ski_imread(spinefile)
mip_mask=imgio.ski_imread(mipmaskfile)
mip_spine_label=imgio.ski_imread(mipspinefile)


2. caculate statistic

In [58]:
import pandas as pd
from spinelib.utils import measure
from skimage.measure import label, regionprops, regionprops_table

#area
arr=measure.label_series_statics(all_spine_label,all_spine_label,measure="area")
df_volume=arr*zpixel*xpixel*ypixel
df_volume.rename(columns={0:'bg',1:'den'},index={0:"area"},inplace=True)
df_volume=pd.DataFrame(df_volume)

tmip_spine_label=np.array([mip_spine_label])
df_max_volume=measure.label_series_statics(tmip_spine_label,tmip_spine_label,measure="area")

#inten
'''
first row : roi
second row : roi aream
other row : mean itensity
'''
same_spine_label=np.array([mip_spine_label for im in imgs])
arr=measure.label_series_statics(imgs,same_spine_label,measure="intensity_mean")
backvs=[np.mean(im[ms==0]) for im,ms in zip(imgs,all_mask)] # mean bg for each frame
denvs=[np.mean(im[ms==1]) for im,ms in zip(imgs,all_mask)] # mean den for each frame
# bg
arr=np.array(arr)
arr[:,0]=backvs
arr[:,1]=denvs
arr=np.row_stack((df_max_volume,arr))
df_mean=pd.DataFrame(arr)
df_mean.rename(columns={0:'bg',1:'den'},index={0:"inten"},inplace=True)

# save csv result
df_volume.to_csv(areafile)
df_mean.to_csv(meanfile)
print("save :",imgfile)
print("save :",mipimgfile)

shape (80, 512, 512) (80, 512, 512)
shape (1, 512, 512) (1, 512, 512)
shape (80, 512, 512) (80, 512, 512)
save : F:\data\dataset\myspine-dataset2\result-time\Stablized tboaltp 20180128n2\Stablized tboaltp 20180128n2.tif
save : F:\data\dataset\myspine-dataset2\result-time\Stablized tboaltp 20180128n2\tMIP-Stablized tboaltp 20180128n2.tif


In [15]:
df_mean

,bg,den,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
area,None,None,20,43,11,9,60,33,13,6,...,14,5,27,47,44,6,15,31,22,18
1,298.793952,3533.607714,772.45,1333.860465,1089.272727,978.666667,1197.116667,839.848485,1378.615385,1429.666667,...,1262.071429,984.0,1157.444444,1088.12766,1312.25,1257.666667,1369.6,1357.741935,1344.681818,1391.555556
2,301.557667,3510.193749,681.6,1332.0,1227.181818,943.222222,1217.033333,916.636364,1406.615385,1522.166667,...,1175.928571,1363.2,1142.814815,1060.93617,1387.227273,1295.0,1572.866667,1488.548387,1394.636364,1204.555556
3,299.354134,3519.402673,849.65,1316.72093,1204.727273,928.111111,1279.616667,937.787879,1322.307692,1377.833333,...,1051.5,1412.6,1337.074074,1062.744681,1257.227273,1133.666667,1686.133333,1333.0,1259.636364,1276.833333
4,300.252916,3563.6398,944.4,1381.860465,1233.090909,1055.0,1329.116667,1226.666667,1214.230769,1523.166667,...,917.071429,1610.0,1221.518519,1162.914894,1178.409091,1392.833333,1859.533333,1371.741935,1459.090909,1390.055556
5,301.067282,3660.065462,961.4,1482.069767,1218.545455,998.333333,1325.55,1123.363636,1245.615385,1492.0,...,1056.214286,1220.8,1315.518519,1061.085106,1290.204545,1614.833333,1613.4,1367.193548,1337.636364,1128.888889
6,303.385143,3685.611783,1000.45,1448.069767,935.727273,1088.666667,1322.566667,1127.515152,1300.230769,1666.0,...,1362.0,1263.0,1291.666667,1203.723404,1184.931818,1913.333333,1655.8,1400.903226,1448.545455,995.555556
7,303.296949,3653.148754,959.3,1505.186047,834.454545,1061.222222,1276.016667,1090.212121,1333.769231,1654.166667,...,1369.5,1112.8,1251.555556,1155.404255,1243.795455,1826.666667,1551.933333,1376.064516,1522.636364,1331.277778
8,304.474246,3558.806418,882.35,1455.837209,982.545455,1046.888889,1259.433333,1118.636364,1368.923077,1489.833333,...,1442.5,1166.6,1234.851852,1211.489362,1204.477273,1491.5,1536.066667,1301.741935,1545.909091,1159.777778
9,305.372766,3524.018269,1028.4,1469.604651,876.545455,1002.888889,1247.05,938.787879,1245.384615,1405.0,...,1383.0,1187.4,1124.777778,1168.957447,1186.454545,1502.666667,1665.2,1285.16129,1591.045455,1094.388889


# 3. Part three ： spine to dendrite

1.modify den skeleton

In [90]:
#--- BINARY---#
img=np.max(imgs,axis=0)
# img=morphology.white_tophat(img,morphology.cube(5))
global_th=binary_func(img)
adth=img>global_th
# adth=local_threshold(imgs[0],offset=100)
noise_mean=np.mean(img[~adth])
img=img-noise_mean
img[img<0]=0
img.shape
all_mask=imgio.ski_imread(maskfile)
from skimage.morphology import skeletonize_3d,binary_dilation
den=skeletonize_3d(np.max(all_mask==1,axis=0))
den=den>0
structure=np.ones((5,)*2)
den=binary_dilation(den,structure)
den=den=skeletonize_3d(den)
viewer=imgio.plot.imshow3d(img,label_lay=[den],
                           label_lay_name=["den"],
                                 )

Exception in callback BaseAsyncIOLoop._handle_events(19228, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(19228, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

In [91]:
viewer=imgio.plot.imshow3d(img,label_lay=[den],
                           label_lay_name=["den"],
                                 )
den=viewer.layers["den"].data

Exception in callback BaseAsyncIOLoop._handle_events(17516, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(17516, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

In [92]:
import scipy.ndimage as ndi

from skimage import morphology
den=den>0
structure=np.ones((3,)*2)
den=morphology.binary_dilation(den,structure)
den=den=skeletonize_3d(den)
def removenode(skeleton,connection):
    for i in range(1,skeleton.shape[0]-1):
        for j in range(1,skeleton.shape[1]-1):
            if cflow.skeleton_leaf.get_border_connection(skeleton,i,j)>=connection:
                skeleton[i,j]=0
removenode(den,3)
den,_=ndi.label(den,np.ones((3,)*2))
viewer=imgio.plot.imshow3d(img,label_lay=[den],
                           label_lay_name=["den"],
                                 )

Exception in callback BaseAsyncIOLoop._handle_events(20108, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(20108, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

2. save dendrite skeleton

In [93]:
tifffile.imwrite(
    skeletonfile,
    den.astype(np.uint16),
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'YX'},
)


3. get trace from spine to dendrite

In [5]:
#--- BINARY---#
all_mask=imgio.ski_imread(maskfile)
den=imgio.ski_imread(skeletonfile)
img=np.max(imgs,axis=0)
global_th=binary_func(img)
adth=img>global_th
# adth=local_threshold(imgs[0],offset=100)
noise_mean=np.mean(img[~adth])
same_spine_maskes=imgio.ski_imread(mipspinefile)
same_spine_maskes[den>0]=1
cords,linemask=spines_distance(img,same_spine_maskes,[20,20])


..\spinelib\utils\spine_struct.py:191: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  cropimg=img[bindbox].copy()
..\spinelib\utils\spine_struct.py:192: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  croplable=labels[bindbox]
..\spinelib\utils\spine_struct.py:210: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  linemas

In [8]:
pointers=[cord[0] for k,cord in cords.items() if cord]
# print(pointers)
vectors=[(cord[0],np.array(cord[1])-cord[0]) for k,cord in cords.items() if cord[1]]
viewer=imgio.plot.imshow3d(img,label_lay=[all_mask,linemask,den],
                           label_lay_name=["label","line","den"],
                           point_lay=[pointers]
                                 )
viewer.add_vectors(
            vectors
        )

<Vectors layer 'Vectors' at 0x2b522d41bb0>

Exception in callback BaseAsyncIOLoop._handle_events(8540, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(8540, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData\R

4. caculate density

In [96]:
import collections
cnts=collections.defaultdict(int)
for k,cord in cords.items():
    if cord[1]:
        x,y=cord[1]
        cnts[den[x,y]]+=1 

dlabels,dareas,indexs=spinelib.utils.measure.label_statics(den)
for lab,area in zip(dlabels,dareas):
    print(lab,area*0.16,cnts[lab],cnts[lab]/area/0.16)

1 72.8 95 1.304945054945055
3 75.2 94 1.25
5 40.160000000000004 44 1.0956175298804782
